# Processing Transcription Data

## Setup

### Installing Packages

In [0]:
%pip install langchain
%pip install openai
%pip install --force-reinstall typing-extensions==4.5.0
%pip install tiktoken

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


### Importing Packages

In [0]:
### OpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI

### Callbacks
from langchain.callbacks import get_openai_callback

### Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Prompt Templates
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

### Python
import pandas as pd
import time
import os
import re
from datetime import datetime as dt
import datetime

### Setting Environment Variables 

In [0]:
os.environ['OPENAI_EMBEDDING_DEPLOYMENT_NAME'] = "t-e-ada-002"
os.environ['OPENAI_EMBEDDING_MODEL_NAME'] = "text-embedding-ada-002"
os.environ['OPENAI_API_BASE'] = "https://gsk-ds.openai.azure.com/"
os.environ['OPENAI_API_KEY'] = "3599efaa37624ebbbe06dcb0fd571622"
os.environ['OPENAI_API_TYPE'] = "Azure"
os.environ['OPENAI_API_VERSION'] = "2023-07-01-preview"
os.environ['OPENAI_DEPLOYMENT_NAME'] = "ds-gpt-4"
os.environ['OPENAI_MODEL_NAME'] = "gpt-4"
os.environ['OPENAI_DEPLOYMENT_NAME_3_5'] = "ds-gpt-35"
os.environ['OPENAI_MODEL_NAME_3_5'] = "gpt-35-turbo"

## Instantiating Generative AI Components

### Instantiating LLM's

In [0]:
# Azure Chat LLM
chat_llm = AzureChatOpenAI(
    deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
    temperature=0,
    verbose=True,
)

# Azure LLM
llm = AzureOpenAI(
    deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
    model_name=os.getenv("OPENAI_MODEL_NAME"),
    temperature=0,
)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:152: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://gsk-ds.openai.azure.com/ to https://gsk-ds.openai.azure.com/openai.
  warnings.warn(
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:159: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expe

### Creating Text Splitter

In [0]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000,
    chunk_overlap=0,
)

## Functions

### Mapping Functions

In [0]:
def check_status(row):
    if "GSO" in row.values:
        return "GSO"
    elif "Discussed" in row.values:
        return "Discussed"
    else:
        return "Not Discussed"

### Ordering Functions

In [0]:
def order_by_suffix(df, column):
    return (
        df.assign(Index=df[column].str.split("_").str[-1].astype(int))
        .sort_values(by=["Index"])
        .drop(columns=["Index"])
        .reset_index(drop=True)
    )

### Share of Voice Functions

In [0]:
def calculate_speaking_sessions(df):

    df['speaking_duration'] = df['end_time'] - df['start_time']
    df['words_per_section'] = df['text'].apply(lambda x: len(x.split( )))

     # Initialize a list to store the aggregated results
    aggregated_data = []

    # Initialize variables to track the current speaker and total time
    current_speaker = None
    total_time = pd.Timedelta(0)
    total_words = 0 

    # Iterate through rows to aggregate time for the same speaker
    for index, row in df.iterrows():
        speaker = row['speaker']
        speaking_duration = row['speaking_duration']
        num_words = row['words_per_section']

        if current_speaker is None:
            current_speaker = speaker

        if speaker == current_speaker:
            total_time += speaking_duration
            total_words += num_words
            
        else:
            aggregated_data.append({'speaker': current_speaker, 'total_time': total_time, 'total_words': total_words})
            current_speaker = speaker
            total_time = speaking_duration
            total_words = num_words

    # Add the last speaker's total time
    if current_speaker is not None:
        aggregated_data.append({'speaker': current_speaker, 'total_time': total_time, 'total_words': total_words})

    # Create a DataFrame from the aggregated data
    result_df = pd.DataFrame(aggregated_data)

    return result_df

In [0]:
def add_silence(transcript_df, threshold=0):
    # Sort the dataframe by start time
    transcript_df.sort_values(by=['start_time'], inplace=True)

    # Initialize an empty list to store rows with gaps
    rows_with_gaps = []

    # Iterate through the rows in the dataframe
    for index, row in transcript_df.iterrows():
        # Check if there's a gap with the next row
        if index < len(transcript_df) - 1:
            current_end_time = row['end_time']
            next_start_time = transcript_df.loc[index + 1, 'start_time']

            # Calculate the time gap
            if next_start_time > current_end_time:
                time_gap = next_start_time - current_end_time
            else:
                time_gap = datetime.timedelta(seconds=0)

            # If there's a gap, add a blank row
            if time_gap > datetime.timedelta(seconds=threshold):  # Adjust the threshold as needed
                blank_row = {
                    'start_time': current_end_time,
                    'end_time': next_start_time,
                    'speaker': 'Silence',
                    'text': ' '
                }
                rows_with_gaps.append(blank_row)

    # Concatenate the original dataframe with the rows containing gaps
    if rows_with_gaps:
        transcript_df = pd.concat([transcript_df] + [pd.DataFrame(rows_with_gaps)])
    
    # Reset the index of the new dataframe
    transcript_df.sort_values(by=['start_time'], inplace=True)
    transcript_df.reset_index(drop=True, inplace=True)

    return transcript_df

In [0]:
def flatten_categorical_metrics(df, categorical_column, dictionary):
    for metric in df.drop(categorical_column, axis=1).columns.to_list():
        for speaker, value in zip(df[categorical_column],df[metric]):
            new_col_name = f'{speaker} {metric}'
            dictionary[new_col_name] = [value]
        
    return dictionary

## Defining Chat Prompts

### Transcript Analysing

In [0]:
discussed_agreed_action = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                You are a helpful assistant that helps to understand if action planning topics were not discussed, discussed or agreed upon in sales calls.
                - You will be given a transcript of sales calls and the action planning topics
                - Your goal is to work out if a action planning topic was discussed or agreed upon in the sales calls

                DEFINITIONS:
                - Not Discussed: The action planning topic was not discussed in the sales call
                - Discussed: The action planning topic was discussed in the sales call
                - Agreed: The action planning topic was agreed upon in the sales call / the hcp already has the action planning topic in place or will be putting it in place

                ACTION PLANNING TOPICS:
                	
                1. Identify Eligible Patients: Searching and creating lists of eligible patients based on the NIP to be vaccinated with Shingrix
                2. Recall Eligible Patients: Inviting patients to be vaccinated to the clinic (e.g., phone call, text message etc..)
                3. Set up Shingrix Clinic: Setting up a dedicated clinic for patients to be vaccinated
                4. Order Shingles Doses: Ordering Shingrix doses to the practice

                TEMPLATE:
                [Number] [Action planning topics] : [Verdict]
                EXAMPLE:
                1. Identify Eligible Patients: Searching and creating lists of eligible patients based on the NIP to be vaccinated with Shingrix : Discussed
                2. Recall Eligible Patients: Inviting patients to be vaccinated to the clinic (e.g., phone call, text message etc..) : Discussed
                3. Set up Shingrix Clinic: Setting up a dedicated clinic for patients to be vaccinated : Not Discussed
                4. Order Shingles Doses: Ordering Shingrix doses to the practice : Agreed
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

find_how_many_times_key_topics_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                You are a helpful assistant that helps to understand if key message topics were conveyed in sales calls.
                - You will be given a transcript of sales calls and the key message topics
                - Your goal is to work out how many times each key message was conveyed by the REP
                
                IMPORTANT: IF THE PORTION OF TEXT HAS MULTIPLE KEY MESSAGES IN IT, COUNT THEM ALL
                IMPORTANT: THERE IS NO LIMIT TO THE LENGTH OF A KEY MESSAGE
                IMPORTANT: RETURN ALL THE KEY MESSAGE COUNTS IN THE SAME ORDER AS THE KEY MESSAGE TOPICS

                KEY MESSAGE TOPICS:
                	
                1. Shingles is a painful disease that can have serious and long-lasting complications
                2. Post-herpetic neuralgia (PHN) affects up to 30% of patients ≥50 years old and is characterised by long-lasting nerve pain
                3. 1 in 4 people in the UK will suffer from shingles in their lifetime	
                4. 90% of adults in the UK are infected with VZV, which causes shingles	
                5. The risk of shingles increases as immune system function declines
                6. More than a rash, shingles and PHN can significantly impact a patient’s quality of life
                7. Burden of Disease
                8. National Immunisation Programme

                TEMPLATE:
                [Number] [Key Message/s] : [Count]
                EXAMPLE:
                1. Shingles is a painful disease that can have serious and long-lasting complications : 0
                2. Post-herpetic neuralgia (PHN) affects up to 30% of patients ≥50 years old and is characterised by long-lasting nerve pain : 1
                3. 1 in 4 people in the UK will suffer from shingles in their lifetime : 1
                4. 90% of adults in the UK are infected with VZV, which causes shingles : 0
                5. The risk of shingles increases as immune system function declines : 4
                6. More than a rash, shingles and PHN can significantly impact a patient’s quality of life : 6
                7. Burden of Disease : 2
                8. National Immunisation Programme : 0
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

objection_raised = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                You are a helpful assistant that helps to understand if objections were raised in sales calls.
                - You will be given a transcript of sales calls and the common objections that are raised in sales calls
                - Your goal is to work out if the objection was raised in the sales calls

                DEFINITIONS:
                - Objection: A reason why a HCP might not want to prescribe a drug or is having difficulty prescribing a drug

                OBJECTIONS:
                
                Objections to do with:
                1. The NIP being confusing
                2. Setting up Shingrix clinics
                3. Patient recall challenges
                4. Second dose compliance


                TEMPLATE:
                [Number] [Objection] : [Verdict]
                EXAMPLE:
                1. The NIP being confusing : True
                2. Setting up Shingrix clinics : False
                3. Patient recall challenges : True
                4. Second dose compliance : True
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

consent_message_delivered = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                You are a helpful assistant that helps to understand if consent message was stated at the start of a sales call.
                - You will be given a transcript of sales calls
                - Your goal is to work out if the consent message was stated at the start of a sales call.

                TEMPLATE:
                [Consent] : [Verdict]
                EXAMPLE:
                Consent : True
                Consent : False
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

## Analysing

In [0]:
CALL_RECORDINGS = [
    # "Oct_20_2023_Call_01",
    # "Oct_20_2023_Call_02",
    # "Oct_20_2023_Call_03",
    # "Oct_20_2023_Call_04",
    # "Oct_20_2023_Call_05",
    # "Oct_20_2023_Call_07",
    # "Oct_20_2023_Call_08",
    # "Oct_20_2023_Call_09",
    # "Oct_20_2023_Call_10",
    # "Oct_20_2023_Call_11",
    # "Oct_20_2023_Call_12",
    # "Oct_27_2023_Call_01",
    # "Oct_27_2023_Call_02",
    # "Oct_27_2023_Call_03",
    # "Oct_27_2023_Call_04",
    # "Oct_27_2023_Call_05",
    # "Oct_27_2023_Call_06",
    # "Oct_27_2023_Call_07",
    # "Oct_27_2023_Call_08",
    # "Oct_27_2023_Call_09",
    # "Oct_27_2023_Call_10",
    # "Oct_27_2023_Call_11",
    # "Oct_27_2023_Call_12",
    # "Nov_03_2023_Call_01",
    # "Nov_03_2023_Call_02",
    # "Nov_03_2023_Call_03",
    # "Nov_03_2023_Call_04",
    # "Nov_03_2023_Call_05",
    # "Nov_03_2023_Call_06",
    # "Nov_03_2023_Call_07",
    # "Nov_03_2023_Call_08",
    # "Nov_10_2023_Call_01",
    # "Nov_10_2023_Call_02",
    # "Nov_10_2023_Call_03",
    # "Nov_10_2023_Call_04",
    # "Nov_10_2023_Call_06",
    # "Nov_17_2023_Call_01",
    # "Nov_17_2023_Call_02",
    # "Nov_17_2023_Call_03",
    # "Nov_17_2023_Call_04",
    # "Nov_17_2023_Call_05",
    # "Nov_17_2023_Call_06",
    # "Nov_17_2023_Call_07",
    # "Nov_17_2023_Call_08",
    # "Nov_24_2023_Call_01",
    # "Nov_24_2023_Call_02",
    # "Nov_24_2023_Call_03",
    # "Nov_24_2023_Call_04",
    # "Nov_24_2023_Call_05",
    # "Nov_24_2023_Call_06",
    # "Nov_24_2023_Call_07",
    # "Dec_01_2023_Call_01",
    # "Dec_01_2023_Call_02",
    # "Dec_01_2023_Call_03",
    # "Dec_01_2023_Call_04",
    # "Dec_01_2023_Call_05",
    # "Dec_01_2023_Call_06",
    # "Dec_01_2023_Call_07",
    # "Dec_01_2023_Call_08",
    # "Dec_01_2023_Call_09",
    # "Dec_01_2023_Call_10",
    # "Dec_01_2023_Call_11",
    # "Dec_01_2023_Call_12",
    # "Dec_01_2023_Call_13",
    # "Dec_01_2023_Call_15",
    # "Dec_08_2023_Call_01",
    # "Dec_08_2023_Call_02",
    # "Dec_08_2023_Call_03",
    # "Dec_08_2023_Call_04",
    # "Dec_08_2023_Call_05",
    # "Dec_08_2023_Call_06",
    # "Dec_08_2023_Call_07",
    # "Dec_08_2023_Call_08",
    # "Dec_08_2023_Call_09",
    # "Dec_08_2023_Call_10",
    # "Dec_08_2023_Call_11",
    # "Dec_08_2023_Call_12",
    # "Dec_08_2023_Call_13",
    # "Dec_15_2023_Call_01",
    # "Dec_15_2023_Call_02",
    # "Dec_15_2023_Call_03",
    # "Dec_15_2023_Call_04",
    # "Dec_15_2023_Call_05",
    # "Dec_15_2023_Call_06",
    # "Dec_15_2023_Call_07",
    # "Dec_15_2023_Call_08",
    # "Dec_15_2023_Call_09",
    # "Dec_15_2023_Call_10",
    # "Dec_22_2023_Call_01",
    # "Dec_22_2023_Call_02",
    # "Dec_22_2023_Call_03",
    # "Dec_22_2023_Call_04",
    # "Dec_22_2023_Call_05",
    # "Dec_22_2023_Call_06",
    # "Dec_22_2023_Call_07",
    # "Dec_22_2023_Call_08",
    # "Dec_22_2023_Call_09",
    # "Jan_12_2024_Call_01",
    # "Jan_12_2024_Call_02",
    "Jan_12_2024_Call_03",
    "Jan_12_2024_Call_04",
    "Jan_12_2024_Call_05",
    "Jan_12_2024_Call_06",
    "Jan_12_2024_Call_07",
    "Jan_12_2024_Call_08",
    "Jan_12_2024_Call_09",
    # "Jan_19_2024_Call_01",
    # "Jan_19_2024_Call_02",
    # "Jan_19_2024_Call_03",
    # "Jan_19_2024_Call_04",
    # "Jan_19_2024_Call_05",
    # "Jan_19_2024_Call_06",
    # "Jan_19_2024_Call_07",
    # "Jan_19_2024_Call_08",
    # "Jan_19_2024_Call_09",
    # "Jan_19_2024_Call_10",
    # "Jan_19_2024_Call_11",
    # "Jan_19_2024_Call_12",
]



base_path = "/dbfs/FileStore/smart_call_insights/max"
folder_path = "jan_12_2024"

### Action Points Analysis

In [0]:
input_relative_path = f"{base_path}/06_section/{folder_path}"
output_relative_path = f"{base_path}/07_action_plan_verdict/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()
        with open(f"{input_relative_path}/{file_name}", "r") as f:
            text = f.read()

        name = os.path.splitext(file_name)[0]
        print(f"Processing {name}")

        chunked_text = text_splitter.create_documents([text])
        action_plan_verdict = dict()
        action_plan_verdict["Call Name"] = match
        action_plan_verdict["Call Number"]= int(match.split("_")[-1])
        date = "_".join(match.split("_")[:3])
        call_datetime = dt.strptime(date, "%b_%d_%Y")
        action_plan_verdict["Call Day"] = call_datetime.strftime("%d")
        action_plan_verdict["Call Month"] = call_datetime.strftime("%m")
        action_plan_verdict["Call Year"] = call_datetime.strftime("%Y")
        total_tokens = 0
        prompt_tokens = 0
        completion_tokens = 0
        successful_requests = 0
        total_cost = 0
        for i, chunk in enumerate(chunked_text):
            with get_openai_callback() as cb:
                output = chat_llm(
                    discussed_agreed_action.format_messages(text=chunk.page_content)
                )
                # print(output.content)
                pattern = r"\d+. (.+): (Discussed|Agreed|Not Discussed)"

                count_list = output.content.split("\n")

                for category in count_list:
                    re_match = re.findall(pattern, category)[0]

                    action_plan = re_match[0].strip().split(":")[0]
                    verdict = re_match[1].strip()

                    if action_plan_verdict.get(action_plan) is None:
                        action_plan_verdict[action_plan] = verdict
                    elif action_plan_verdict.get(action_plan) == "Not Discussed":
                        if verdict == "Discussed":
                            action_plan_verdict[action_plan] = "Discussed"
                        elif verdict == "Agreed":
                            action_plan_verdict[action_plan] = "Agreed"
                    elif action_plan_verdict.get(action_plan) == "Discussed":
                        if verdict == "Agreed":
                            action_plan_verdict[action_plan] = "Agreed"
                    elif action_plan_verdict.get(action_plan) == "Agreed":
                        pass

                # Costs and Tokens
                total_tokens += cb.total_tokens
                prompt_tokens += cb.prompt_tokens
                completion_tokens += cb.completion_tokens
                successful_requests += cb.successful_requests
                total_cost += cb.total_cost

        df = pd.DataFrame(action_plan_verdict, index=[0])
        df = df.replace({"Agreed": "GSO"})
        df["Selling Outcome"] = df.apply(check_status, axis=1)
        df.to_csv(
            f"{output_relative_path}/{match}_action_plan_verdict.csv", index=False
        )

        print(
            f"Tokens Used: {total_tokens}, Prompt Tokens: {prompt_tokens}, Completion Tokens: {completion_tokens}, Successful Requests: {successful_requests} \nTotal Cost (USD): ${round(total_cost, 2)}"
        )
        print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")

Processing Nov03_callrecording_01_sectioned
Tokens Used: 8998, Prompt Tokens: 8788, Completion Tokens: 210, Successful Requests: 2 
Total Cost (USD): $0.28
Time Taken: 12.18 seconds
Processing Nov03_callrecording_02_sectioned
Tokens Used: 2230, Prompt Tokens: 2125, Completion Tokens: 105, Successful Requests: 1 
Total Cost (USD): $0.07
Time Taken: 7.65 seconds
Processing Nov03_callrecording_03_sectioned
Tokens Used: 3110, Prompt Tokens: 3004, Completion Tokens: 106, Successful Requests: 1 
Total Cost (USD): $0.1
Time Taken: 5.96 seconds
Processing Nov03_callrecording_04_sectioned
Tokens Used: 5203, Prompt Tokens: 4990, Completion Tokens: 213, Successful Requests: 2 
Total Cost (USD): $0.16
Time Taken: 16.36 seconds
Processing Nov03_callrecording_05_sectioned
Tokens Used: 5671, Prompt Tokens: 5458, Completion Tokens: 213, Successful Requests: 2 
Total Cost (USD): $0.18
Time Taken: 14.48 seconds
Processing Nov03_callrecording_06_sectioned
Tokens Used: 8875, Prompt Tokens: 8666, Completio

### Analysing Key Topics

In [0]:
input_relative_path = f"{base_path}/06_section/{folder_path}"
output_relative_path = f"{base_path}/07_key_topics_conveyed/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()
        with open(f"{input_relative_path}/{file_name}", "r") as f:
            text = f.read()

        name = os.path.splitext(file_name)[0]
        print(f"Processing {name}")

        chunked_text = text_splitter.create_documents([text])
        key_topic_count = dict()
        key_topic_count["Call Name"] = match
        key_topic_count["Call Number"]= int(match.split("_")[-1])
        date = "_".join(match.split("_")[:3])
        call_datetime = dt.strptime(date, "%b_%d_%Y")
        key_topic_count["Call Day"] = call_datetime.strftime("%d")
        key_topic_count["Call Month"] = call_datetime.strftime("%m")
        key_topic_count["Call Year"] = call_datetime.strftime("%Y")
        total_tokens = 0
        prompt_tokens = 0
        completion_tokens = 0
        successful_requests = 0
        total_cost = 0
        for i, chunk in enumerate(chunked_text):
            with get_openai_callback() as cb:
                output = chat_llm(
                    find_how_many_times_key_topics_template.format_messages(
                        text=chunk.page_content
                    )
                )

                pattern = r"\d+.(.+): (\d+)"

                count_list = output.content.split("\n")

                for category in count_list:
                    re_match = re.findall(pattern, category)[0]

                    key_topic = re_match[0].strip()
                    count = int(re_match[1].strip())

                    if key_topic_count.get(key_topic):
                        key_topic_count[key_topic] += count
                    else:
                        key_topic_count[key_topic] = count

                # Costs and Tokens
                total_tokens += cb.total_tokens
                prompt_tokens += cb.prompt_tokens
                completion_tokens += cb.completion_tokens
                successful_requests += cb.successful_requests
                total_cost += cb.total_cost

        df = pd.DataFrame(key_topic_count, index=[0])
        df.to_csv(
            f"{output_relative_path}/{match}_key_topic_count.csv", index=False
        )

        print(
            f"Tokens Used: {total_tokens}, Prompt Tokens: {prompt_tokens}, Completion Tokens: {completion_tokens}, Successful Requests: {successful_requests} \nTotal Cost (USD): ${round(total_cost, 2)}"
        )
        print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")

Processing Nov03_callrecording_01_sectioned
Tokens Used: 9326, Prompt Tokens: 8992, Completion Tokens: 334, Successful Requests: 2 
Total Cost (USD): $0.29
Time Taken: 24.8 seconds
Processing Nov03_callrecording_02_sectioned
Tokens Used: 2394, Prompt Tokens: 2227, Completion Tokens: 167, Successful Requests: 1 
Total Cost (USD): $0.08
Time Taken: 11.63 seconds
Processing Nov03_callrecording_03_sectioned
Tokens Used: 3273, Prompt Tokens: 3106, Completion Tokens: 167, Successful Requests: 1 
Total Cost (USD): $0.1
Time Taken: 8.81 seconds
Processing Nov03_callrecording_04_sectioned
Tokens Used: 5528, Prompt Tokens: 5194, Completion Tokens: 334, Successful Requests: 2 
Total Cost (USD): $0.18
Time Taken: 25.33 seconds
Processing Nov03_callrecording_05_sectioned
Tokens Used: 5996, Prompt Tokens: 5662, Completion Tokens: 334, Successful Requests: 2 
Total Cost (USD): $0.19
Time Taken: 19.91 seconds
Processing Nov03_callrecording_06_sectioned
Tokens Used: 9204, Prompt Tokens: 8870, Completio

### Objection Raised

In [0]:
input_relative_path = f"{base_path}/06_section/{folder_path}"
output_relative_path = f"{base_path}/07_objection_raised/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()
        with open(f"{input_relative_path}/{file_name}", "r") as f:
            text = f.read()

        name = os.path.splitext(file_name)[0]
        print(f"Processing {name}")

        chunked_text = text_splitter.create_documents([text])
        objection_verdict = dict()
        objection_verdict["Call Name"] = match
        objection_verdict["Call Number"]= int(match.split("_")[-1])
        date = "_".join(match.split("_")[:3])
        call_datetime = dt.strptime(date, "%b_%d_%Y")
        objection_verdict["Call Day"] = call_datetime.strftime("%d")
        objection_verdict["Call Month"] = call_datetime.strftime("%m")
        objection_verdict["Call Year"] = call_datetime.strftime("%Y")
        total_tokens = 0
        prompt_tokens = 0
        completion_tokens = 0
        successful_requests = 0
        total_cost = 0
        for i, chunk in enumerate(chunked_text):
            with get_openai_callback() as cb:
                output = chat_llm(
                    objection_raised.format_messages(text=chunk.page_content)
                )
                pattern = r"\d+. (.+): (True|False)"

                count_list = output.content.split("\n")

                for category in count_list:
                    re_match = re.findall(pattern, category)[0]

                    key_message = re_match[0].strip()
                    verdict = re_match[1].strip()

                    if objection_verdict.get(key_message) is None:
                        objection_verdict[key_message] = verdict
                    elif objection_verdict.get(key_message) == "False":
                        if verdict == "True":
                            objection_verdict[key_message] = "True"
                    elif objection_verdict.get(key_message) == "True":
                        pass

                # Costs and Tokens
                total_tokens += cb.total_tokens
                prompt_tokens += cb.prompt_tokens
                completion_tokens += cb.completion_tokens
                successful_requests += cb.successful_requests
                total_cost += cb.total_cost

        df = pd.DataFrame(objection_verdict, index=[0])
        df = df.replace({"True": "Objection Raised", "False": "Objection Not Raised"})
        df.to_csv(
            f"{output_relative_path}/{match}_objection_verdict.csv", index=False
        )

        print(
            f"Tokens Used: {total_tokens}, Prompt Tokens: {prompt_tokens}, Completion Tokens: {completion_tokens}, Successful Requests: {successful_requests} \nTotal Cost (USD): ${round(total_cost, 2)}"
        )
        print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")

Processing Nov03_callrecording_01_sectioned
Tokens Used: 8496, Prompt Tokens: 8424, Completion Tokens: 72, Successful Requests: 2 
Total Cost (USD): $0.26
Time Taken: 7.56 seconds
Processing Nov03_callrecording_02_sectioned
Tokens Used: 1979, Prompt Tokens: 1943, Completion Tokens: 36, Successful Requests: 1 
Total Cost (USD): $0.06
Time Taken: 3.48 seconds
Processing Nov03_callrecording_03_sectioned
Tokens Used: 2858, Prompt Tokens: 2822, Completion Tokens: 36, Successful Requests: 1 
Total Cost (USD): $0.09
Time Taken: 2.85 seconds
Processing Nov03_callrecording_04_sectioned
Tokens Used: 4698, Prompt Tokens: 4626, Completion Tokens: 72, Successful Requests: 2 
Total Cost (USD): $0.14
Time Taken: 6.61 seconds
Processing Nov03_callrecording_05_sectioned
Tokens Used: 5166, Prompt Tokens: 5094, Completion Tokens: 72, Successful Requests: 2 
Total Cost (USD): $0.16
Time Taken: 4.79 seconds
Processing Nov03_callrecording_06_sectioned
Tokens Used: 8374, Prompt Tokens: 8302, Completion Token

### Share of Voice

In [0]:
input_relative_path = f"{base_path}/05_share_of_voice/{folder_path}"
output_relative_path = f"{base_path}/07_share_of_voice/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()
        df = pd.read_csv(os.path.join(input_relative_path, file_name))

        name = os.path.splitext(file_name)[0]
        print(f"Processing {name}")

        df["start_time"] = pd.to_timedelta(df["start_time"])
        df["end_time"] = pd.to_timedelta(df["end_time"])

        silence_df = add_silence(df)

        
        speaking_sessions = calculate_speaking_sessions(df)
        silence_sessions = calculate_speaking_sessions(silence_df)

        speaker_sessions_agg = speaking_sessions.groupby('speaker').agg({
            'total_time': ['count', "sum", "mean"],
            'total_words': ['sum', "mean"],}).reset_index()
        
        silence_sessions_agg = silence_sessions.groupby('speaker').agg({
            'total_time': ['count', "sum", "mean"]}).reset_index()
        
        speaker_sessions_agg.columns = ['Speaker', 'Times Spoken', 'Total Speaking Time', "Average Speaking Time", 'Number of words', "Average Number of words"]
        silence_sessions_agg.columns = ['Speaker', 'Times Spoken', 'Total Speaking Time', "Average Speaking Time"]

        silence_only_sessions_agg = silence_sessions_agg[silence_sessions_agg['Speaker'] == 'Silence'].reset_index(drop=True)

        silence_sessions_agg['Percentage of Speaking Time'] = silence_sessions_agg['Total Speaking Time'] / silence_sessions_agg['Total Speaking Time'].sum()

        percentage_of_speaking_time = silence_sessions_agg[['Speaker', 'Percentage of Speaking Time']]

        share_of_voice = dict()
        share_of_voice["Call Name"] = match
        share_of_voice["Call Number"]= int(match.split("_")[-1])
        date = "_".join(match.split("_")[:3])
        call_datetime = dt.strptime(date, "%b_%d_%Y")
        share_of_voice["Call Day"] = call_datetime.strftime("%d")
        share_of_voice["Call Month"] = call_datetime.strftime("%m")
        share_of_voice["Call Year"] = call_datetime.strftime("%Y")

        # print(share_of_voice)

        share_of_voice = flatten_categorical_metrics(speaker_sessions_agg , 'Speaker', share_of_voice)
        share_of_voice = flatten_categorical_metrics(silence_only_sessions_agg , 'Speaker', share_of_voice)
        share_of_voice = flatten_categorical_metrics(percentage_of_speaking_time , 'Speaker', share_of_voice)

        metric_columns = list(share_of_voice.keys())[5:]
        speaker_set = set()
        metric_set = set()

        metric_columns = list(share_of_voice.keys())[5:]
        speaker_set = set()
        metric_set = set()

        for column_name in metric_columns:
            speaker = column_name.split()[0]
            metric = " ".join(column_name.split()[1:])
            speaker_set.add(speaker)
            metric_set.add(metric)

        speaker_list = sorted(list(speaker_set))
        metric_list = sorted(list(metric_set), reverse=True)

        column_order = [f"{speaker} {metric}" for speaker in speaker_list for metric in metric_list]
        column_order = [col for col in column_order if not ('Silence' in col and 'words' in col)]

        full_column_order = ['Call Name', 'Call Number', 'Call Day', 'Call Month', 'Call Year'] + column_order

        share_of_voice_df = pd.DataFrame(share_of_voice, columns=full_column_order)

        share_of_voice_df.to_csv(
            f"{output_relative_path}/{match}_share_of_voice.csv", index=False
        )

        print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")

Processing Oct20_callrecording_01_sov
Time Taken: 0.16 seconds
Processing Oct20_callrecording_02_sov
Time Taken: 0.11 seconds
Processing Oct20_callrecording_03_sov
Time Taken: 0.09 seconds
Processing Oct20_callrecording_04_sov
Time Taken: 0.35 seconds
Processing Oct20_callrecording_05_sov
Time Taken: 0.33 seconds
Processing Oct20_callrecording_07_sov
Time Taken: 0.19 seconds
Processing Oct20_callrecording_08_sov
Time Taken: 0.11 seconds
Processing Oct20_callrecording_09_sov
Time Taken: 0.09 seconds
Processing Oct20_callrecording_10_sov
Time Taken: 0.21 seconds
Processing Oct20_callrecording_11_sov
Time Taken: 0.23 seconds
Processing Oct20_callrecording_12_sov
Time Taken: 0.15 seconds


### Consent Message Delivered

In [0]:
input_relative_path = f"{base_path}/06_section/{folder_path}"
output_relative_path = f"{base_path}07_consent_message_delivered/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()
        with open(f"{input_relative_path}/{file_name}", "r") as f:
            text = f.read()

        name = os.path.splitext(file_name)[0]
        print(f"Processing {name}")

        chunked_text = text_splitter.create_documents([text])
        consent_verdict = dict()
        consent_verdict["Call Name"] = match
        consent_verdict["Call Number"]= int(match.split("_")[-1])
        date = "_".join(match.split("_")[:3])
        call_datetime = dt.strptime(date, "%b_%d_%Y")
        consent_verdict["Call Day"] = call_datetime.strftime("%d")
        consent_verdict["Call Month"] = call_datetime.strftime("%m")
        consent_verdict["Call Year"] = call_datetime.strftime("%Y")
        total_tokens = 0
        prompt_tokens = 0
        completion_tokens = 0
        successful_requests = 0
        total_cost = 0
        for i, chunk in enumerate(chunked_text[:1]):
            with get_openai_callback() as cb:
                output = chat_llm(
                    consent_message_delivered.format_messages(text=chunk.page_content)
                )

            pattern = r"(Consent) : (True|False)"
            re_match = re.findall(pattern, output.content)[0]

            verdict = re_match[1].strip()

            if verdict == 'True':
                consent_verdict['Consent'] = True
            else:
                consent_verdict['Consent'] = False

            # Costs and Tokens
            total_tokens += cb.total_tokens
            prompt_tokens += cb.prompt_tokens
            completion_tokens += cb.completion_tokens
            successful_requests += cb.successful_requests
            total_cost += cb.total_cost

        df = pd.DataFrame(consent_verdict, index=[0])
        df.to_csv(
            f"{output_relative_path}/{match}_consent_verdict.csv", index=False
        )

        print(
            f"Tokens Used: {total_tokens}, Prompt Tokens: {prompt_tokens}, Completion Tokens: {completion_tokens}, Successful Requests: {successful_requests} \nTotal Cost (USD): ${round(total_cost, 2)}"
        )

        print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")
            
            

## Combining Data

In [0]:
action_plan_relative_path = f"{base_path}/07_action_plan_verdict/{folder_path}"
key_topic_relative_path = f"{base_path}/07_key_topics_conveyed/{folder_path}"
objection_relative_path = f"{base_path}/07_objection_raised/{folder_path}"
share_of_voice_relative_path = f"{base_path}/07_share_of_voice/{folder_path}"
output_relative_path = f"{base_path}/08_results/{folder_path}"

action_plan_combined_df = pd.DataFrame()
key_topic_combined_df = pd.DataFrame()
objection_combined_df = pd.DataFrame()
share_of_voice_combined_df = pd.DataFrame()

for file_name in sorted(os.listdir(action_plan_relative_path)):
    if any((match := x) in file_name for x in CALL_RECORDINGS):
        start_time = time.time()

        action_plan_df = pd.read_csv(
            f"{action_plan_relative_path}/{match}_action_plan_verdict.csv"
        )
        key_topic_df = pd.read_csv(
            f"{key_topic_relative_path}/{match}_key_topic_count.csv"
        )
        objection_df = pd.read_csv(
            f"{objection_relative_path}/{match}_objection_verdict.csv"
        )
        share_of_voice_df = pd.read_csv(
            f"{share_of_voice_relative_path}/{match}_share_of_voice.csv"
        )

        action_plan_combined_df = pd.concat([action_plan_combined_df, action_plan_df])
        key_topic_combined_df = pd.concat([key_topic_combined_df, key_topic_df])
        objection_combined_df = pd.concat([objection_combined_df, objection_df])
        share_of_voice_combined_df = pd.concat([share_of_voice_combined_df, share_of_voice_df])

action_plan_combined_df = action_plan_combined_df.sort_values(by=['Call Month', 'Call Day', 'Call Number'])
key_topic_combined_df = key_topic_combined_df.sort_values(by=['Call Month', 'Call Day', 'Call Number'])
objection_combined_df = objection_combined_df.sort_values(by=['Call Month', 'Call Day', 'Call Number'])
share_of_voice_combined_df = share_of_voice_combined_df.sort_values(by=['Call Month', 'Call Day', 'Call Number'])

action_plan_combined_df.to_csv(
    f"{output_relative_path}/action_plan_combined.csv", index=False
)
key_topic_combined_df.to_csv(
    f"{output_relative_path}/key_topic_combined.csv", index=False
)
objection_combined_df.to_csv(
    f"{output_relative_path}/objection_combined.csv", index=False
)
share_of_voice_combined_df.to_csv(
    f"{output_relative_path}/share_of_voice.csv", index=False
)

print(f"Time Taken: {round(time.time() - start_time, 2)} seconds")

Time Taken: 0.13 seconds


In [0]:
pd.read_csv(f"{output_relative_path}/share_of_voice.csv")

,Call Name,Call Number,Call Day,Call Month,HCP Total Speaking Time,HCP Times Spoken,HCP Percentage of Speaking Time,HCP Number of words,HCP Average Speaking Time,HCP Average Number of words,REP Total Speaking Time,REP Times Spoken,REP Percentage of Speaking Time,REP Number of words,REP Average Speaking Time,REP Average Number of words,Silence Total Speaking Time,Silence Times Spoken,Silence Percentage of Speaking Time,Silence Average Speaking Time,Unknown Total Speaking Time,Unknown Times Spoken,Unknown Percentage of Speaking Time,Unknown Number of words,Unknown Average Speaking Time,Unknown Average Number of words
0,Oct20_callrecording_01,1,20,10,0 days 00:03:33.990000,18,0.430737,691,0 days 00:00:11.888333333,38.388889,0 days 00:03:28.190000,17,0.419062,698,0 days 00:00:12.246470588,41.058824,0 days 00:01:14.620000,38,0.150201,0 days 00:00:01.963684210,NaN,NaN,NaN,NaN,NaN,NaN
1,Oct20_callrecording_02,2,20,10,0 days 00:00:35.720000,9,0.098465,107,0 days 00:00:03.968888888,11.888889,0 days 00:05:06.700000,9,0.845439,952,0 days 00:00:34.077777777,105.777778,0 days 00:00:19.350000,16,0.053340,0 days 00:00:01.209375,0 days 00:00:01,1.0,0.002757,7.0,0 days 00:00:01,7.0
2,Oct20_callrecording_03,3,20,10,0 days 00:01:26.900000,11,0.248669,267,0 days 00:00:07.900000,24.272727,0 days 00:04:12.740000,12,0.723230,734,0 days 00:00:21.061666666,61.166667,0 days 00:00:09.820000,12,0.028100,0 days 00:00:00.818333333,NaN,NaN,NaN,NaN,NaN,NaN
3,Oct20_callrecording_04,4,20,10,0 days 00:03:23.210000,12,0.299159,610,0 days 00:00:16.934166666,50.833333,0 days 00:06:16.950000,13,0.554934,1036,0 days 00:00:28.996153846,79.692308,0 days 00:01:39.110000,60,0.145907,0 days 00:00:01.651833333,NaN,NaN,NaN,NaN,NaN,NaN
4,Oct20_callrecording_05,5,20,10,0 days 00:21:40.690000,76,0.495250,4431,0 days 00:00:17.114342105,58.302632,0 days 00:16:02.170000,78,0.366355,3155,0 days 00:00:12.335512820,40.448718,0 days 00:05:58.990000,228,0.136689,0 days 00:00:01.574517543,0 days 00:00:04.480000,4.0,0.001706,16.0,0 days 00:00:01.120000,4.0
5,Oct20_callrecording_07,7,20,10,0 days 00:06:58.760000,26,0.256046,1310,0 days 00:00:16.106153846,50.384615,0 days 00:16:16.780000,24,0.597240,2854,0 days 00:00:40.699166666,118.916667,0 days 00:03:58.130000,124,0.145602,0 days 00:00:01.920403225,0 days 00:00:01.820000,2.0,0.001113,4.0,0 days 00:00:00.910000,2.0
6,Oct20_callrecording_08,8,20,10,0 days 00:00:50.660000,17,0.100363,158,0 days 00:00:02.980000,9.294118,0 days 00:06:48.100000,18,0.808487,1151,0 days 00:00:22.672222222,63.944444,0 days 00:00:46.010000,41,0.091150,0 days 00:00:01.122195121,NaN,NaN,NaN,NaN,NaN,NaN
7,Oct20_callrecording_09,9,20,10,0 days 00:00:33.770000,7,0.107627,119,0 days 00:00:04.824285714,17.000000,0 days 00:04:24.550000,8,0.843134,750,0 days 00:00:33.068750,93.750000,0 days 00:00:15.450000,16,0.049240,0 days 00:00:00.965625,NaN,NaN,NaN,NaN,NaN,NaN
8,Oct20_callrecording_10,10,20,10,0 days 00:12:37.030000,39,0.633291,2152,0 days 00:00:19.411025641,55.179487,0 days 00:03:05.720000,38,0.155364,620,0 days 00:00:04.887368421,16.315789,0 days 00:04:12.220000,125,0.210994,0 days 00:00:02.017760,0 days 00:00:00.420000,1.0,0.000351,1.0,0 days 00:00:00.420000,1.0
9,Oct20_callrecording_11,11,20,10,0 days 00:10:58.760000,61,0.395942,2099,0 days 00:00:10.799344262,34.409836,0 days 00:11:56.100000,58,0.430405,2202,0 days 00:00:12.346551724,37.965517,0 days 00:04:43.750000,166,0.170545,0 days 00:00:01.709337349,0 days 00:00:05.170000,4.0,0.003107,12.0,0 days 00:00:01.292500,3.0
